In [1]:
# import pandas as pd
import MagmaPandas as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import meltInc.plotting as p
from functools import partial
from MagmaPandas.geochemistry.fO2 import fO2_QFM
from MagmaPandas.geochemistry.Kd_ol_melt import Kd_FeMg_vectorised
from MagmaPandas.geochemistry.Fe_redox import Fe_redox
from MagmaPandas.geochemistry.PEC import Fe_equilibrate

p.layout()
config = mp.configuration()
config.print()

Kd Fe-Mg ol-melt.......................toplis
Melt Fe3+/Fe2+........................borisov
Melt thermometer...............putirka2008_14
Volatile solubility model......IaconoMarziano


In [2]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')
inclusions = melt.iloc[0:10,:]

In [7]:
inclusion = melt.iloc[0,:].copy()


In [10]:
# inclusion.drop("CaO", inplace=True)
inclusion.recalculate()
inclusion.weights

SiO2      60.084
Al2O3    101.961
MgO       40.304
FeO       71.844
Na2O      61.979
K2O       94.195
MnO       70.937
TiO2      79.865
P2O5     141.943
Cr2O3    151.989
CO2       44.008
H2O       18.015
F         18.998
S         32.065
Cl        35.453
Name: weight, dtype: float64

In [6]:
inclusion["total"]

88.82662

In [ ]:
forsterite_host = 0.9
P_bar = 1e3

In [ ]:
w, F1, oc = melt.Fe_equilibrate(forsterite_host, P_bar)


In [ ]:
o_crystallised = []
for _, inclusion in melt.iterrows():
    *_, OC = Fe_equilibrate(inclusion, forsterite_host, P_bar)
    o_crystallised.append(OC)
o_crystallised

In [ ]:
oc

In [ ]:
o